<a href="https://colab.research.google.com/github/ArtemKarDev/nlp_test_bewise/blob/main/pars_nlp_test_bewise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Необходимо написать скрипт для парсинга диалогов из файла test_data.csv. Получившийся скрипт необходимо выложить в гит репозиторий и прислать ссылку в качестве результата прохождения тестового задания. Данные выкладывать в гит не следует. 
2. Главные задачи, которые должен выполнять скрипт:
- Извлекать реплики с приветствием – где менеджер поздоровался. 
- Извлекать реплики, где менеджер представил себя. 
- Извлекать имя менеджера. 
- Извлекать название компании. 
- Извлекать реплики, где менеджер попрощался.
- Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

3. Рекомендации:
- Сделать локальную копию файла test_data.csv, в исходнике никакие данные не менять!
- Можно создать дополнительное поле в таблице test_data.csv, куда будет сохраняться результат парсинга – например, напротив реплики в столбце “insight” можно ставить флаг того, что эта реплика с приветствием greeting=True.
- Для выполнения задачи можно использовать любые библиотеки и NLP модели. 
- Попробуйте учесть возможные синонимичные выражения, которые могут помочь с извлечением данных сущностей. 

Результат выполнения задачи отправить на почту hr@bewise.ru, также указать свое ФИО и ссылку на резюме.

Работы принимаются до 04.09.2022, далее в течении 7 календарных дней мы анализируем результаты и приглашаем на собеседования подходящих кандидатов.


План решения:

1. Подготовка данных

2. Выбор инструмента для определения приветсвенных и прощальных сообщений внутри каждого диалога

3. Анализ полученных результатов

4. Подведение итогов

## Подготовка данных

In [2]:
# загружаю таблицу с моего гугл диска по расшеренной ссылке
!wget 'https://drive.google.com/uc?id=18uUAWLYNeINvzoNDLlKkvlZ99f_PxWo4' -O test_data.csv

--2022-08-25 18:50:32--  https://drive.google.com/uc?id=18uUAWLYNeINvzoNDLlKkvlZ99f_PxWo4
Resolving drive.google.com (drive.google.com)... 172.253.115.138, 172.253.115.139, 172.253.115.101, ...
Connecting to drive.google.com (drive.google.com)|172.253.115.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-14-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0vkl9vntsoasjs66m9u233i49popcl8m/1661453400000/00838955718561688027/*/18uUAWLYNeINvzoNDLlKkvlZ99f_PxWo4?uuid=7e8f4d3d-7881-4fb1-9136-e299967a410b [following]
--2022-08-25 18:50:32--  https://doc-00-14-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0vkl9vntsoasjs66m9u233i49popcl8m/1661453400000/00838955718561688027/*/18uUAWLYNeINvzoNDLlKkvlZ99f_PxWo4?uuid=7e8f4d3d-7881-4fb1-9136-e299967a410b
Resolving doc-00-14-docs.googleusercontent.com (doc-00-14-docs.googleusercontent.com)... 172.217.0.33, 2607:f8b0:4004:800::2001
Connecting to do

In [3]:
# импорт библиотек
import numpy as np
import pandas as pd
import re

# для векторизации текста
from sklearn.feature_extraction.text import CountVectorizer
# загрузим библиотеку для расчетов похожести
from sklearn.metrics.pairwise import cosine_similarity

!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 5.4 MB/s 
     |████████████████████████████████| 8.2 MB 55.7 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=82a87b1c2863530a90b7a65a8470980f4f8cc9c80f5db7a0d140a54d361e1cce
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [4]:
!pip install natasha
from natasha import (Segmenter,
                     MorphVocab,
                     NamesExtractor,

                     NewsEmbedding,
                     NewsMorphTagger,
                     NewsSyntaxParser,
                     NewsNERTagger,

                     PER,
                     Doc)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.4 MB 1.3 MB/s 
     |████████████████████████████████| 41 kB 145 kB/s 
     |████████████████████████████████| 49 kB 7.2 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=3eb6faa8fb1a46fd8d9329f42b9760d4c98bbe2dcb84f816783c0bc20ef29ad4
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [5]:
pd.set_option('max_rows', 99)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]


In [6]:
data = pd.read_csv('test_data.csv')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


In [8]:
data.head(10)

,dlg_id,line_n,role,text
0,0,0,manager,Алло
1,0,1,client,Алло здравствуйте
2,0,2,manager,Добрый день
3,0,3,client,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
4,0,4,manager,Ага
5,0,5,client,Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать
6,0,6,manager,Да мы работаем с компанией которая нам подливает поэтому спасибо огромное
7,0,7,manager,Как как бы уже до этого момента работаем все устраивает + у нас сопровождение поэтому
8,0,8,client,Угу а на что вы обращаете внимание при выборе
9,0,9,manager,Как бы нет


Выполнена загрузка CSV файлов в DataFrame. Дат отправки сообщений нет, сортировка выполнена по индексам внутри каждого диалога.

In [9]:
data['dlg_id'].value_counts()

5    143
0    109
2     85
1     55
3     53
4     35
Name: dlg_id, dtype: int64

В нашем распоряжении всего 6 диалогов с клиентами с количеством реплик от 35 до 143 сообщений. Суммарно 480 записей. 

Судя по всему диалоги были распознаны из аудиозвонков.

Задача требует в основном анализа сообщений от менеджера - для контроля приветсвий и прощаний. А из клиентских сообщений требуется выделить только наименование компании.



При чтение первых 10 строк заметил что 
- даилоговые роли перепутаны - должно бы так что менеджер совершает звинки клиентам. А в первом диалоге в реплике с индексом 3 явно говорит не клиент.Поменяю роли местами.
- все имена и наименования с маленькой буквы


In [10]:
def change_role(data):
  if data == 'manager':
    return 'client'
  return 'manager'

data['role'] = data['role'].apply(lambda x: change_role(x))

Проведем предобработку текстов сообщений:
 - переведем в нижний регистр,
 - оставляем только русские буквы, пробелы и тире.
 - заменим несколько пробелов на один пробел.
 - очистим от частотных/не значимых словосочетаний
 - токенизируем и лемматизируем

In [11]:
def filter_text(text):
    """
    принимает текст,
    на выходе обработанны текст
    """
    text = str(text)

    text = text.lower()
    #
    #if len()
    #text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'(.+[.]jpg)|(.+[.]pdf)', '', text)
    text = [c for c in text if c in 'абвгдеёжзийлкмнопрстуфхцчшщъыьэюя- ']
    text = ''.join(text)
    return text

def lemmatizer(text):
    lemm_text = []
    for word in text.split():
        p = morph.parse(word)[0]
        lemm_text.append(p.normal_form)
    return ' '.join(lemm_text)

In [12]:
data['filtered_text'] = data['text'].apply(lambda x: filter_text(x))
data['filtered_text'] = data['filtered_text'].apply(lambda x: lemmatizer(x))
data[['text','filtered_text']]

,text,filtered_text
0,Алло,алло
1,Алло здравствуйте,алло здравствуйте
2,Добрый день,добрый день
3,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мы с серый у вы скоро срок заканчиваться
4,Ага,ага
...,...,...
475,По поводу виджетов и с ними уже обсудите конкретно продам,по повод виджет и с они уже обсудить конкретно продать
476,Все я вам высылаю счет и с вами на связи если будут вопросы можете писать на ватсапе,всё я вы высылать счёт и с вы на связь если быть вопрос мочь писать на ватсап
477,Спасибо спасибо,спасибо спасибо
478,Да да тогда созвонимся ага спасибо вам давайте,да да тогда созвониться ага спасибо вы давать


Произвели очистку и лемматизацию текста

можно приступать к векторизации

Создадим строки с которыми будем искать похожесть, можно было бы сделать просто поиск по вхождению нужного слова в сообщение, но я решил искать сходство реплик менеджера с эталонной строкой приветствия с помощью косинусного расстояния.

Проблема - чем длинее строка реплики тем больше расстояние и необходимо подбирать порог. А также наличие слова например - "Добрый", является ли это полноценным приветсвием, возможно это вопрос к корпоративной культуре и к должностным инструкциям менеджера. Я же решил что буду рассматривать только полноценное приветсвие , поэтому разделил фразы приветсвий на две группы - для векторизации биграммой и монограммой.

Проблема- при соединении векторизованных матриц необходимо соблюсти размерность, либо добавлять пустые значения.

In [13]:
# Создадим строки с которыми будем искать похожесть
# список со списками - чем длинее фраза тем длинее список
# 
greeting_words = [['привет','здравствуйте'], ['добрый день','день добрый']]
greeting_words2 = ['добрый день','день добрый']
parting_words = [['досвидания','',''], ['до свидания','хорошего дня',' хорошего вечера']]
parting_words2 = ['до свидания','хорошего дня',' хорошего вечера']
#introduced_words = ['','','']


In [14]:
wordtovec1 = CountVectorizer(ngram_range=(1,1))
wordtovec2 = CountVectorizer(ngram_range=(2,2))

def get_matrix(data_fit, data):
  text_matrix1 = wordtovec1.fit_transform(data_fit)
  words_matrix1 = wordtovec1.transform(data[0])
  
  text_matrix2 = wordtovec2.fit_transform(data_fit)
  words_matrix2 = wordtovec2.transform(data[1])

  text_matrix = np.column_stack((text_matrix1.toarray(), text_matrix2.toarray()))
  words_matrix = np.column_stack((words_matrix1.toarray(), words_matrix2.toarray()))
  return text_matrix, words_matrix


## Менеджер приветствовал

In [15]:
# передадим в функцию расчета векторизированнную матрицу  
# и сравниваться она будет со созданой строкой 
#  - для расчета похожести 
words_matrix, greeting_words_matrix = get_matrix( data['filtered_text'] , greeting_words)
cosine_sim_w2v = cosine_similarity(words_matrix, greeting_words_matrix)
#cosine_sim_w2v.shape

In [16]:
# добавим столбец с отобраным из вектора похожести максимальным значением 
# и колонку с логическими значениямим требуемой задачей - greeting

data['greeting_vec'] = cosine_sim_w2v.tolist()
data['greeting'] = data['greeting_vec'].apply(lambda x: True if  max(x) > 0.01 else None)
# сохраним индексы этих реплик
greeting_indx = data[data['role'] == 'manager'].loc[~data['greeting'].isna()].index.to_list()

In [17]:
data.iloc[greeting_indx]

,dlg_id,line_n,role,text,filtered_text,greeting_vec,greeting
1,0,1,manager,Алло здравствуйте,алло здравствуйте,"[0.0, 0.5773502691896258]",True
110,1,1,manager,Алло здравствуйте,алло здравствуйте,"[0.0, 0.5773502691896258]",True
166,2,2,manager,Алло здравствуйте,алло здравствуйте,"[0.0, 0.5773502691896258]",True
250,3,1,manager,Алло дмитрий добрый день,алло дмитрий добрый день,"[0.3779644730092272, 0.0]",True


## Менеджер попрощался

In [18]:
# Лемматизирую фразы
for i in range(0,len(parting_words)):
  for j in range(0,len(parting_words[i])):
    parting_words[i][j] = lemmatizer(parting_words[i][j])
parting_words

[['досвидание', '', ''], ['до свидание', 'хороший день', 'хороший вечер']]

In [19]:
words_matrix, parting_words_matrix = get_matrix( data['filtered_text'] , parting_words)
cosine_sim_w2v = cosine_similarity(words_matrix, parting_words_matrix)
data['parting_vec'] = cosine_sim_w2v.tolist()
data['parting'] = data['parting_vec'].apply(lambda x: True if  max(x) > 0.01 else None)
# сохраним индексы этих реплик
parting_indx = data[data['role'] == 'manager'].loc[~data['parting'].isna()].index.to_list()

In [20]:
data.iloc[parting_indx]

,dlg_id,line_n,role,text,filtered_text,greeting_vec,greeting,parting_vec,parting
108,0,108,manager,Всего хорошего до свидания,весь хороший до свидание,"[0.0, 0.0]",None,"[0.3779644730092272, 0.0, 0.0]",True
163,1,54,manager,До свидания,до свидание,"[0.0, 0.0]",None,"[0.5773502691896258, 0.0, 0.0]",True
335,4,33,manager,Во вторник все ну с вами да тогда до вторника до свидания,в вторник всё ну с вы да тогда до вторник до свидание,"[0.0, 0.0]",None,"[0.20851441405707477, 0.0, 0.0]",True
479,5,142,manager,Ну до свидания хорошего вечера,ну до свидание хороший вечер,"[0.0, 0.0]",None,"[0.3333333333333333, 0.0, 0.3333333333333333]",True


## Менеджер представился

для данной задачи воспользуюсь библиотекой Natasha - решающая базовые задачи НЛП по русскому языку, а точнее одним из её инструментов для извлечения именованных сущностей из русских текстов.

С учетом того что принято представляться в начале разговора - имена будем искать в первых 4 репликах диалога от лица менеджера. Также необходимо учесть что в репликах есть имена к кому обращается менеджер, соответсвенно необходимо отобрать те реплики в которых менеджер сообщаяет своё имя - в данном тексте это происходит с использованием слов: 'меня зовут' и 'это'


In [21]:
# инициализируем объекты 
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()

morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

# doc = Doc(text)
# doc.segment(segmenter)
# doc.tag_morph(morph_tagger)
# doc.parse_syntax(syntax_parser)
# doc.tag_ner(ner_tagger)

In [22]:
# сложная функция, написана практически методом 'тыка'
# но у неё получается определять имена в словах с маленькой буквы
# а затем она капитализирует слово и проверяет на наличие тагга PER(персона) 

def get_name(data):

  list_name = []
  doc = Doc(data)
  macth = names_extractor(data)
  for i in macth:
    if i.fact.first:
      list_name.append(i.fact.first)
  if list_name:
    jam = []
    for name in list_name:
      doc = Doc(name.capitalize())
      doc.segment(segmenter)
      doc.tag_ner(ner_tagger)  
      for item in doc.spans:
        if item.type == 'PER':
          #print(item)
          jam.append(item.text)
    if len(jam) > 0:
      return jam

In [30]:
intro_words = ['зовут','это']
lemm_intro_words = [lemmatizer(i) for i in intro_words]

def where_intro(text):
  for i in lemm_intro_words:
    if i in text.lower().split():
      return True
  return False 


In [36]:
# отберем индексы первых фраз менеджера в каждом диалоге
first_rep_manag = data[(data['line_n'] < 4) & (data['role'] == 'manager')].index.to_list()
# 
intro_rep_manag = data.loc[first_rep_manag]['filtered_text'].apply(lambda x: where_intro(x))
intro_rep_manag

1      False
3       True
110    False
111     True
112    False
166    False
167     True
250    False
251     True
305    False
338     True
340    False
Name: filtered_text, dtype: bool

In [37]:
data['name'] = None
data.loc[first_rep_manag, 'name'] = data.loc[first_rep_manag,'filtered_text'].apply(lambda x: get_name(x))
data[~(data['name'].isna()) & intro_rep_manag]

,dlg_id,line_n,role,text,filtered_text,greeting_vec,greeting,parting_vec,parting,name
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мы с серый у вы скоро срок заканчиваться,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,[Ангелина]
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,я звать ангелина компания диджитал бизнес звонить вы по повод продление а мы сель обратить внимание что у вы срок заканчиваться,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,[Ангелина]
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мастер мы с вы сотрудничать по видео там,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,[Ангелина]
251,3,2,manager,Добрый меня максим зовут компания китобизнес удобно говорить,добрый я максим звать компания китобизнес удобно говорить,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,[Максим]
338,5,1,manager,Да это анастасия,да это анастасия,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,[Анастасия]


In [25]:
name_indx = data[(data['role'] == 'manager') & (~data['name'].isna())] .index.to_list()

In [26]:
data.iloc[name_indx]

,dlg_id,line_n,role,text,filtered_text,greeting_vec,greeting,parting_vec,parting,name
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мы с серый у вы скоро срок заканчиваться,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,[Ангелина]
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,я звать ангелина компания диджитал бизнес звонить вы по повод продление а мы сель обратить внимание что у вы срок заканчиваться,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,[Ангелина]
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мастер мы с вы сотрудничать по видео там,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,[Ангелина]
250,3,1,manager,Алло дмитрий добрый день,алло дмитрий добрый день,"[0.3779644730092272, 0.0]",True,"[0.0, 0.0, 0.0]",None,[Дмитрий]
251,3,2,manager,Добрый меня максим зовут компания китобизнес удобно говорить,добрый я максим звать компания китобизнес удобно говорить,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,[Максим]
338,5,1,manager,Да это анастасия,да это анастасия,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,[Анастасия]


## Наименование оргнизации

In [ ]:
def return_org(data):
  doc = Doc(data)
  doc.segment(segmenter)
  doc.tag_ner(ner_tagger)
  jam=list()
  for item2 in doc.spans:
    if item2.type == 'ORG':
      print(item2)
      jam.append(item2.text)

  return str(jam)

In [ ]:
data[data['role'] == 'manager'].drop('filtered_text', axis=1).groupby('dlg_id').head(2)

,dlg_id,line_n,role,text,greeting_vec,greeting,parting_vec,parting,name
1,0,1,manager,Алло здравствуйте,"[0.0, 0.5773502691896258]",True,"[0.0, 0.0, 0.0]",None,None
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,[ангелина]
110,1,1,manager,Алло здравствуйте,"[0.0, 0.5773502691896258]",True,"[0.0, 0.0, 0.0]",None,None
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,[ангелина]
166,2,2,manager,Алло здравствуйте,"[0.0, 0.5773502691896258]",True,"[0.0, 0.0, 0.0]",None,None
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,[ангелина]
250,3,1,manager,Алло дмитрий добрый день,"[0.3779644730092272, 0.0]",True,"[0.0, 0.0, 0.0]",None,[дмитрий]
251,3,2,manager,Добрый меня максим зовут компания китобизнес удобно говорить,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,[максим]
305,4,3,manager,Вот по виду платежи пообщаться помните мы вот с вами обсуждали вас там 1 функционал не очень в нашем продукте подходил вот по промежуточным задачам вот а скажите пожалуйста а сейчас этот вопрос актуален для вас,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,None
308,4,6,manager,Я понял ну мы хотели бы просто предложить такой вариант развития событий в принципе мы,"[0.0, 0.0]",None,"[0.0, 0.0, 0.0]",None,None


In [ ]:
data[(~(data['name'].isna())) & (data['role'] == 'manager')][['dlg_id','line_n','text','role','name']]

,dlg_id,line_n,text,role,name
3,0,3,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,manager,[ангелина]
111,1,2,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,manager,[ангелина]
130,1,21,Да конечно вам просто нужно будет зайти в настройки а мы серым а убрать там пользователей а после этого написать в техподдержку а маш что вот вы убрали подтвердите что сейчас вам,manager,[маш]
167,2,3,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,manager,[ангелина]
205,2,41,Группировка полей да,manager,[поле]
228,2,64,Сапаров собачка,manager,[сапар]
229,2,65,А вот астана собачка ираклас точка кз,manager,[астана]
239,2,75,У вас в любое время есть с какого по какого,manager,[любой]
241,2,77,А в любое время хорошо тогда я аналитику по этому вопросу напишу а он мне тоже напишет свои окошки а я вам в ватсап,manager,[любой]
250,3,1,Алло дмитрий добрый день,manager,[дмитрий]


In [ ]:
python3 -m spacy download ru_core_web_md

SyntaxError: ignored

In [ ]:
nlp = spacy.load("en_core_web_md")
search = nlp("payment date")
text_to_be_searched = nlp("Party A will pay Party B on the transaction date.")
threshold = 0.8

matched_words = []
for token in text_to_be_searched:
    print(token, token.similarity(search))
    if token.similarity(search) > threshold:
        matched_words.append(token)

print(f"\nMatched words: {matched_words}")

In [ ]:
!pip install transformers

import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('Добрый день', model, tokenizer).shape)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(312,)


In [ ]:
a = embed_bert_cls('привет', model, tokenizer)
b = embed_bert_cls('здравствуйте', model, tokenizer)
c = embed_bert_cls('вечер', model, tokenizer)
1-cosine(a, b) , 1- cosine(a, c)

(0.8861705660820007, 0.844275712966919)

In [ ]:

from scipy.spatial.distance import cosine
cosine([1, 1, 1, 1], [0, 0, 0, 0])

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


0